In [28]:
import pyomo.environ as pyo

In [46]:
# Data Definitions
time_periods = [t for t in range(0,12)]
max_capacity = 1000
charging_efficiency = 0.95
discharging_efficiency = 0.95
min_charge_state = 100
max_charge_state = 950
max_charge_discharge_rate = 250

price_dict = {0:0.05,1:0.04,2:0.04,3:0.03,4:0.03,5:0.04,6:0.06,7:0.08,8:0.10,9:0.12,10:0.14,11:0.15}
demand_dict = {0:300,1:280,2:260,3:250,4:240,5:260,6:300,7:350,8:400,9:450,10:500,11:550}

solar_gen_dict = {0:0,1:0,2:0,3:0,4:0,5:50,6:150,7:300,8:400,9:450,10:480,11:500}
wind_gen_dict = {0:100,1:80,2:50,3:30,4:20,5:40,6:50,7:80,8:120,9:150,10:180,11:200}

In [48]:
total_energy = sum(solar_gen_dict[t] + wind_gen_dict[t] for t in time_periods) + 500
total_demand = sum(demand_dict[t] for t in time_periods)
print('Total Energy:', total_energy)
print('Total Demand:', total_demand)

Total Energy 3930
Total Demand 4140


In [44]:
# Trial and Error
time_periods = [t for t in range(0,3)]
price_dict = {0:0.05,1:0.04,2:0.04}
demand_dict = {0:300,1:280,2:260}
solar_gen_dict = {0:0,1:0,2:0}
wind_gen_dict = {0:100,1:80,2:50}

In [42]:
def battery_model(time_periods,max_capacity,charging_efficiency,discharging_efficiency,min_charge_state,max_charge_state,max_charge_discharge_rate,price_dict,demand_dict,solar_gen_dict,wind_gen_dict):
    
    # Initialize the model
    model = pyo.ConcreteModel(name = 'Microgrid Optimizer')
   
    # Initialize Sets and Parameters 
    model.T = pyo.Set(initialize = time_periods)
    model.Max_Cap = pyo.Param(initialize = max_capacity)
    model.Charge_Eff = pyo.Param(initialize = charging_efficiency)
    model.Discharge_Eff = pyo.Param(initialize = discharging_efficiency)
    model.Min_C_State = pyo.Param(initialize = min_charge_state)
    model.Max_C_State = pyo.Param(initialize = max_charge_state)
    model.Max_Charge_Discharge_Rate = pyo.Param(initialize = max_charge_discharge_rate)
    model.Price = pyo.Param(model.T,initialize = price_dict)
    model.Demand = pyo.Param(model.T,initialize = demand_dict)
    model.Solar_Gen = pyo.Param(model.T,initialize = solar_gen_dict)
    model.Wind_Gen = pyo.Param(model.T,initialize = wind_gen_dict)
    model.Initial_Charge = pyo.Param(initialize = 500)

    # Initialize Variables
    # def initBattery(model,i):
    #     if i == 0:
    #         return 500
    #     else:
    #         pass
    # model.B_State = pyo.Var(model.T, bounds = (model.Min_C_State,model.Max_C_State), initialize = initBattery)
    model.B_State = pyo.Var(model.T, bounds = (0,model.Max_Cap)) # initialize = initBattery
    model.Charge_R = pyo.Var(model.T, bounds = (0,model.Max_Charge_Discharge_Rate))
    model.Discharge_R = pyo.Var(model.T, bounds = (0,model.Max_Charge_Discharge_Rate))
    model.Grid_I = pyo.Var(model.T, within = pyo.NonNegativeReals)
    model.Grid_E = pyo.Var(model.T, within = pyo.NonNegativeReals)
    model.Relative_Flow = pyo.Var(model.T, within = pyo.NonNegativeReals)   # Can be eliminated if import and export constraiints combined
    model.Charge_Binary = pyo.Var(model.T, within = pyo.Binary)
    # Objective Function
    def obj_rule(model):
        return sum(model.Price[t]*(model.Grid_I[t] - model.Grid_E[t]) for t in model.T)
    model.obj = pyo.Objective(rule = obj_rule, sense = pyo.minimize)

    # Constraints
    # 1st Constraint: Battery State
    def batteryConstraint(model, t):
        if t != 0:
            return model.B_State[t] == model.B_State[t-1] + model.Charge_Eff*model.Charge_R[t] - model.Discharge_R[t]/model.Discharge_Eff
        else:
            return model.B_State[t] == model.Initial_Charge + model.Charge_Eff*model.Charge_R[t] - model.Discharge_R[t]/model.Discharge_Eff
    model.state_constr = pyo.Constraint(model.T, rule = batteryConstraint)

    # 2nd Constraint: Min Battery Charge
    def stateMinCharge(model,t):
        return model.B_State[t] >= model.Min_C_State
    model.state_min_c_constr = pyo.Constraint(model.T, rule = stateMinCharge)    

    # 3rd Constraint: Max Battery Charge
    def stateMaxCharge(model,t):
        return model.B_State[t] <= model.Max_C_State
    model.state_max_c_constr = pyo.Constraint(model.T, rule = stateMaxCharge)

    # 4nd Constraint : Grid Import
    def gridImportConstraint(model, t):
        return model.Grid_I[t] == model.Charge_Eff*model.Charge_R[t] + model.Relative_Flow[t]
    model.grid_import_constr = pyo.Constraint(model.T, rule = gridImportConstraint)

    # 5th Constraint : Grid Export
    def gridExportConstraint(model, t):
        return model.Grid_E[t] == model.Discharge_R[t]/model.Discharge_Eff + model.Relative_Flow[t]
    model.grid_export_constr = pyo.Constraint(model.T, rule = gridExportConstraint)

    # 6th Constraint: Source Energy
    def sourceEnergyConstraint(model, t):
        return model.Grid_I[t] <=  model.Solar_Gen[t] + model.Wind_Gen[t]
    model.source_energy_constr = pyo.Constraint(model.T, rule = sourceEnergyConstraint)

    # 7th Constraint: Customer Demand
    def customerDemandConstraint(model, t):
        return model.Grid_E[t] >= model.Demand[t]
    model.customer_demand_constr = pyo.Constraint(model.T, rule = customerDemandConstraint)

    # 8.1th Constraint: Disable simultaneous charge and discharge
    def chargeBinary(model,t):
        return model.Charge_R[t]/model.Max_Charge_Discharge_Rate <= model.Charge_Binary[t]
    model.charge_binary_constr = pyo.Constraint(model.T, rule = chargeBinary)

    #8.2th Constraint: Disable simultaneous charge and discharge
    def dischargeBinary(model,t):
        return model.Discharge_R[t]/model.Max_Charge_Discharge_Rate <= 1 - model.Charge_Binary[t]
    model.discharge_binary_constr = pyo.Constraint(model.T, rule = dischargeBinary)

    # Solve
    solver = pyo.SolverFactory('glpk')
    results = solver.solve(model)
    pyo.assert_optimal_termination(results)
    model.display()

In [43]:
battery_model(time_periods,max_capacity,charging_efficiency, discharging_efficiency,min_charge_state,max_charge_state,max_charge_discharge_rate,price_dict,demand_dict,solar_gen_dict,wind_gen_dict)

Model Microgrid Optimizer

  Variables:
    B_State : Size=2, Index=T
        Key : Lower : Value : Upper  : Fixed : Stale : Domain
          0 :     0 : 300.0 : 1000.0 : False : False :  Reals
          1 :     0 : 100.0 : 1000.0 : False : False :  Reals
    Charge_R : Size=2, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 : 250.0 : False : False :  Reals
          1 :     0 :   0.0 : 250.0 : False : False :  Reals
    Discharge_R : Size=2, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 : 190.0 : 250.0 : False : False :  Reals
          1 :     0 : 190.0 : 250.0 : False : False :  Reals
    Grid_I : Size=2, Index=T
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 : 100.0 :  None : False : False : NonNegativeReals
          1 :     0 :  80.0 :  None : False : False : NonNegativeReals
    Grid_E : Size=2, Index=T
        Key : Lower : Value : Upper : Fixed : Stale 

In [4]:
# Separate Model:
# Initialize the model
model = pyo.ConcreteModel()
   
# Initialize Sets and Parameters 
model.T = pyo.Set(initialize = time_periods)
model.Max_Cap = pyo.Param(initialize = max_capacity)
model.Charge_Eff = pyo.Param(initialize = charging_efficiency)
model.Discharge_Eff = pyo.Param(initialize = discharging_efficiency)
model.Min_C_State = pyo.Param(initialize = min_charge_state)
model.Max_C_State = pyo.Param(initialize = max_charge_state)
model.Max_Charge_Discharge_Rate = pyo.Param(initialize = max_charge_discharge_rate)
model.Price = pyo.Param(model.T,initialize = price_dict)
model.Demand = pyo.Param(model.T,initialize = demand_dict)
model.Solar_Gen = pyo.Param(model.T,initialize = solar_gen_dict)
model.Wind_Gen = pyo.Param(model.T,initialize = wind_gen_dict)
model.Initial_Charge = pyo.Param(initialize = 500)


In [5]:
# Variables
model.B_State = pyo.Var(model.T, bounds = (0,model.Max_Cap)) # initialize = initBattery
model.Charge_R = pyo.Var(model.T, bounds = (0,model.Max_Charge_Discharge_Rate))
model.Discharge_R = pyo.Var(model.T, bounds = (0,model.Max_Charge_Discharge_Rate))
model.Grid_I = pyo.Var(model.T, within = pyo.NonNegativeReals)
model.Grid_E = pyo.Var(model.T, within = pyo.NonNegativeReals)
model.Relative_Flow = pyo.Var(model.T, within = pyo.NonNegativeReals)   # Can be eliminated if import and export constraiints combined
model.Charge_Binary = pyo.Var(model.T, within = pyo.Binary)

In [ ]:
# Objective Function
def obj_rule(model):
    return sum(model.Price[t]*(model.Grid_I[t] - model.Grid_E[t]) for t in model.T)
model.obj = pyo.Objective(rule = obj_rule, sense = pyo.minimize)

In [ ]:
 # Constraints
# 1st Constraint: Battery State
def batteryConstraint(model, t):
    if t != 0:
        return model.B_State[t] == model.B_State[t-1] + model.Charge_Eff*model.Charge_R[t] - model.Discharge_R[t]/model.Discharge_Eff
    else:
        return model.B_State[t] == model.Initial_Charge + model.Charge_Eff*model.Charge_R[t] - model.Discharge_R[t]/model.Discharge_Eff
model.state_constr = pyo.Constraint(model.T, rule = batteryConstraint)

In [ ]:
# 2nd Constraint: Min Battery Charge
def stateMinCharge(model,t):
    return model.B_State[t] >= model.Min_C_State
model.state_min_c_constr = pyo.Constraint(model.T, rule = stateMinCharge)    

In [ ]:
# 3rd Constraint: Max Battery Charge
def stateMaxCharge(model,t):
    return model.B_State[t] <= model.Max_C_State
model.state_max_c_constr = pyo.Constraint(model.T, rule = stateMaxCharge)

In [ ]:
# 4nd Constraint : Grid Import
def gridImportConstraint(model, t):
    return model.Grid_I[t] == model.Charge_Eff*model.Charge_R[t] + model.Relative_Flow[t]
model.grid_import_constr = pyo.Constraint(model.T, rule = gridImportConstraint)

In [ ]:
# 5th Constraint : Grid Export
def gridExportConstraint(model, t):
    return model.Grid_E[t] == model.Discharge_R[t]/model.Discharge_Eff + model.Relative_Flow[t]
model.grid_export_constr = pyo.Constraint(model.T, rule = gridExportConstraint)

In [ ]:
# 6th Constraint: Source Energy
def sourceEnergyConstraint(model, t):
    return model.Grid_I[t] <=  model.Solar_Gen[t] + model.Wind_Gen[t]
model.source_energy_constr = pyo.Constraint(model.T, rule = sourceEnergyConstraint)

In [ ]:
# 7th Constraint: Customer Demand
def customerDemandConstraint(model, t):
    return model.Grid_E[t] >= model.Demand[t]
model.customer_demand_constr = pyo.Constraint(model.T, rule = customerDemandConstraint)

In [ ]:
# 8.1th Constraint: Disable simultaneous charge and discharge
def chargeBinary(model,t):
    return model.Charge_R[t]/model.Max_Charge_Discharge_Rate <= model.Charge_Binary[t]
model.charge_binary_constr = pyo.Constraint(model.T, rule = chargeBinary)

#8.2th Constraint: Disable simultaneous charge and discharge
def dischargeBinary(model,t):
    return model.Discharge_R[t]/model.Max_Charge_Discharge_Rate <= 1 - model.Charge_Binary[t]
model.discharge_binary_constr = pyo.Constraint(model.T, rule = dischargeBinary)